### Importation les biblio

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import keras
from keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model
from keras.optimizers import SGD

### Importation de la data

In [ ]:
df = pd.read_csv('../input/voicegender/voice.csv')

### Pré-traitement et visulisation data

In [ ]:
df.columns


In [ ]:
df.shape

In [ ]:
df.head(100)

In [ ]:
df.info()

In [ ]:
#Checking des valeurs nulles¶
df.isnull().sum()

In [ ]:
print("Total number of labels: {}".format(df.shape[0]))
print("Number of male: {}".format(df[df.label == 'male'].shape[0]))
print("Number of female: {}".format(df[df.label == 'female'].shape[0]))

Il y'aura pas un biais la data est bien equilibré, les voix females sont equivaut aux vois males

In [ ]:
#Checking the correlation between each feature¶
df.corr()

In [ ]:
sns.pairplot(df[['meanfreq', 'Q25', 'Q75', 'skew', 'centroid', 'label','kurt','maxdom']],hue='label', size=2)

D'apres ces information on distingue que maxdom et centroid et kurt sont des variable depndants, alors on doit les supprimer pour ne pas biaiser les calculs

In [ ]:
df=df.drop(['maxdom','centroid','kurt'],axis=1)

Preparer les entrees et la sortie pour conctruire les modeles 

In [ ]:
X=df.drop(['label'],axis=1)
y=df['label']

Binaisriser la sortie

In [ ]:
y=y.map({'male':0,'female':1})

Rendre les entrees numerique dans un meme echelle

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

Répartir les jeu de data en train_data et test_data

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

### Conctruction des modeles

#### Tree

In [ ]:
from sklearn import tree
dtc=tree.DecisionTreeClassifier()
dtc.fit(X_train,y_train)
y_dtc=dtc.predict(X_test)
cm=confusion_matrix(y_test,y_dtc)
print(accuracy_score(y_test,y_dtc))
print(cm)
print(classification_report(y_test,y_dtc))

In [ ]:
sns.heatmap(cm,annot=True)
plt.xlabel('actual')
plt.ylabel('y_predict')

#### Ensemble

In [ ]:
from sklearn import ensemble
rf=ensemble.RandomForestClassifier()
rf.fit(X_train,y_train)
y_rf=rf.predict(X_test)
cm=confusion_matrix(y_test,y_rf)
print(accuracy_score(y_test,y_rf))
print(cm)
print(classification_report(y_test,y_rf))

In [ ]:
sns.heatmap(cm,annot=True)
plt.xlabel('actual')
plt.ylabel('y_predict')

#### SVM

In [ ]:
from sklearn import svm
svm = svm.SVC()
svm.fit(X_train, y_train)
y_svm = svm.predict(X_test)
cm=confusion_matrix(y_test,y_svm)
print(accuracy_score(y_test, y_svm))
print(cm)
print(classification_report(y_test, y_svm))

In [ ]:
sns.heatmap(cm,annot=True)
plt.xlabel('actual')
plt.ylabel('y_predict')

#### Réseau de neurons

In [ ]:
n_cols = X_train.shape[1]
y_cat = to_categorical(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
n_cols = X_train.shape[1]
y_cat = to_categorical(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=1)
hist = History()
model = Sequential()
model.add(Dense(1000, activation='relu', input_dim = n_cols))
model.add(Dense(700, activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(400, activation='relu'))


model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

train=model.fit(X_train, y_train, epochs = 50, validation_split = .1, callbacks = [hist])

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
print(train.history['accuracy'])

In [ ]:
print(train.history['val_accuracy']) 

In [ ]:
def plot_scores(train):
    accuracy=train.history['accuracy']
    val_accuracy=train.history['val_accuracy']
    epochs=range(len(accuracy))
    plt.plot(epochs,accuracy,'b',label='Score apprentissage')
    plt.plot(epochs,val_accuracy,'r',label='Score validation')
    plt.title('Scores')
    plt.legend()
    plt.show()

In [ ]:
plot_scores(train)

#### Conclusion : 